In [9]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')


sys.path.append('../')  # Add the parent directory to the system path
import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies
import utils.haystack_utils as haystack_utils

%reload_ext autoreload
%autoreload 2

In [10]:
# Run overview
model_name = "tiny-stories-2L-33M"
layer_name = "L0"
print_model_name = f"{model_name}-{layer_name}"
df = pd.read_csv(f"{model_name}/wandb_runs.csv")
df = df.sort_values(by="l1_coeff", ascending=True)
df.columns

Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'act', 'batch_size', 'beta1', 'beta2', 'buffer_batches', 'buffer_mult',
       'buffer_size', 'd_mlp', 'data_paths', 'expansion_factor', 'l1_coeff',
       'layer', 'lr', 'model', 'model_batch_size', 'num_eval_batches',
       'num_eval_tokens', 'num_training_tokens', 'seed', 'seq_len',
       'use_wandb', 'wd', 'avg_directions', 'batch', 'bias_mean', 'bias_std',
       'dead_directions', 'epoch', 'l1_loss', 'l2_loss',
       'long term dead directions', 'loss'],
      dtype='object')

In [11]:
fig = px.line(df, x="l1_coeff", y=["l2_loss", "l1_loss", "avg_directions"], markers=True, title=f"{print_model_name}: L1 loss, L2 loss, and average number of active directions")
fig.update_layout(
    xaxis_title="L1 coefficient",
    yaxis_title="",
    legend_title="",
    width = 800,
    xaxis={'tickformat':'.1e'}
)
fig.update_xaxes(type='linear')
fig.show()

In [12]:
prompts = load_tinystories_validation_prompts()

(INFO) 03:25:26: Loaded 21990 TinyStories validation prompts


In [13]:
model = HookedTransformer.from_pretrained(
        model_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        device=device,
    )

Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [22]:
def strip_final_numbers(filename):
    return re.sub(r'_\d+$', '', filename)
    
STRIP_FINAL_NUMBERS = True

def load_encoder(save_name, model_name):
    json_save_name = save_name
    if STRIP_FINAL_NUMBERS:
        json_save_name = strip_final_numbers(save_name)
    with open(f"{model_name}/{json_save_name}.json", "r") as f:
        cfg = json.load(f)

    cfg = AutoEncoderConfig(
        cfg["layer"], cfg["act"], cfg["expansion_factor"], cfg["l1_coeff"]
    )

    if cfg.act_name == "hook_mlp_out":
        d_in = model.cfg.d_model  # d_mlp
    else:
        d_in = model.cfg.d_mlp
    d_hidden = d_in * cfg.expansion_factor

    encoder = AutoEncoder(d_hidden, cfg.l1_coeff, d_in)
    encoder.load_state_dict(torch.load(os.path.join(model_name, save_name + ".pt")))
    encoder.to(device)
    return encoder, cfg

save_names = [f.split(".")[0] for f in os.listdir(model_name) if f.endswith('.pt')]
# encoders = [load_encoder(save_name, model_name) for save_name in save_names]

In [23]:
loss_data = []
for save_name in tqdm(save_names):
    encoder, cfg = load_encoder(save_name, model_name)
    original_loss, encoder_loss, zero_ablation_loss = evaluate_autoencoder_reconstruction(encoder, cfg.encoder_hook_point, prompts[:200], model)
    loss_data.append([cfg.l1_coeff, original_loss, encoder_loss, zero_ablation_loss])
loss_df = pd.DataFrame(loss_data, columns=["L1 coefficient", "Original Loss", "Reconstruction Loss", "Zero Ablation Loss"])
loss_df = loss_df.sort_values(by="L1 coefficient", ascending=True)
loss_df["L1 coefficient"] = loss_df["L1 coefficient"].astype(str)

  0%|          | 0/13 [00:00<?, ?it/s]

18_morning_sun_9999


100%|██████████| 200/200 [00:08<00:00, 23.81it/s]
(INFO) 03:30:30: Average loss increase after encoding: 0.1115


18_morning_sun_19999


100%|██████████| 200/200 [00:06<00:00, 28.92it/s]
(INFO) 03:30:39: Average loss increase after encoding: 0.0995


...

100%|██████████| 200/200 [00:06<00:00, 29.23it/s]
(INFO) 03:31:54: Average loss increase after encoding: 0.0797


18_morning_sun_119999


100%|██████████| 200/200 [00:06<00:00, 29.08it/s]
(INFO) 03:32:02: Average loss increase after encoding: 0.0787


18_morning_sun


100%|██████████| 200/200 [00:06<00:00, 29.17it/s]
(INFO) 03:32:10: Average loss increase after encoding: 0.0794


In [26]:
loss_df = loss_df.melt(id_vars=["L1 coefficient"], var_name="Loss Type", value_name="Loss", value_vars=["Original Loss", "Reconstruction Loss", "Zero Ablation Loss"])
fig = px.line(loss_df, x="L1 coefficient", y="Loss", color="Loss Type", markers=True,  title=f"{print_model_name}: Encoder reconstruction loss increase")
fig.update_layout(
    xaxis_title="L1 coefficient",
    yaxis_title="Loss increase",
    width = 800,
    xaxis={'tickformat':'.1e'}
)
fig.update_xaxes(type='linear')
fig.show()

In [25]:
@torch.no_grad()
def get_acts(prompt: str, model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    _, cache = model.run_with_cache(prompt, names_filter=cfg.encoder_hook_point)
    acts = cache[cfg.encoder_hook_point].squeeze(0)
    _, _, mid_acts, _, _ = encoder(acts)
    return mid_acts

def get_max_activations(prompts: list[str], model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    activations = []
    for prompt in tqdm(prompts):
        acts = get_acts(prompt, model, encoder, cfg)
        max_prompt_activation = acts.max(0)[0]
        activations.append(max_prompt_activation)

    max_activation_per_prompt = torch.stack(activations)  # n_prompt x d_enc

    total_activations = max_activation_per_prompt.sum(0)
    print(f"Active directions on validation data: {total_activations.nonzero().shape[0]} out of {total_activations.shape[0]}")
    return max_activation_per_prompt

def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [24]:
# l1 = 2e-4
# def get_encoder_by_l1(encoders, l1_coeff):
#     for encoder, cfg in encoders:
#         if cfg.l1_coeff == l1_coeff:
#             return encoder, cfg
#     raise ValueError(f"Encoder with L1 coefficient {l1_coeff} not found")
# encoder, cfg = get_encoder_by_l1(encoders, l1)
encoder, cfg = load_encoder("11_generous_sponge", model_name)
print(f"Encoder L1 coefficient: {cfg.l1_coeff}")

18_morning_sun


Encoder L1 coefficient: 0.0001


In [27]:
total_tokens = 0
for prompt in prompts: 
    tokens = model.to_tokens(prompt)
    total_tokens += torch.numel(tokens)
print(total_tokens)

4765918


In [28]:
feature_frequencies = get_encoder_feature_frequencies(prompts, model, encoder, cfg)
zero_activating_features = (feature_frequencies == 0).sum(0).item()
low_density = ((feature_frequencies > 0) & (feature_frequencies < 1e-6)).sum(0).item()
high_density = (feature_frequencies > 1e-6).sum(0).item()
print(zero_activating_features, low_density, high_density)
fig = px.histogram(feature_frequencies.cpu().numpy(), histnorm='probability', title=f"{print_model_name} L1={cfg.l1_coeff}: Histogram of feature frequencies", nbins=40)
fig.update_yaxes(type='log')
fig.update_layout(xaxis_title="Feature frequency", yaxis_title="Probability", showlegend=False, width=600)

  0%|          | 15/21990 [00:00<05:06, 71.78it/s]

100%|██████████| 21990/21990 [05:18<00:00, 69.04it/s]


Number of active features over 4765918 tokens: 16384
Number of average active features per token: 101.56
0 0 16384


No low frequency features
Should we have them?
If yes: 
- L1 too low
- Expansion too low

In [29]:
max_activation_per_prompt = get_max_activations(prompts, model, encoder, cfg)

  0%|          | 0/21990 [00:00<?, ?it/s]

Active directions on validation data: 16384 out of 16384


In [30]:
def plot_direction_frequency(data: list[str], direction: int, cfg: AutoEncoderConfig):
    activations = []
    for prompt in tqdm(data):
        tokens = model.to_tokens(prompt)
        _, cache = model.run_with_cache(
            tokens, names_filter=f"blocks.{cfg.layer}.{cfg.act_name}"
            )
        acts = cache[f"blocks.{cfg.layer}.{cfg.act_name}"].squeeze(0)
        _, _, mid_acts, _, _ = encoder(acts)
        activations.append(mid_acts[:, direction])
    activations = torch.cat(activations)
    print(activations.shape)

    fig = px.histogram(activations.tolist(), 
                       title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for direction {direction}", 
                       histnorm="probability")
    fig.update_layout(
        xaxis_title="Activation",
        yaxis_title="Probability",
        width = 600,
        showlegend=False
    )
    fig.update_yaxes(type='log')
    fig.show()

direction = 0
plot_direction_frequency(prompts[:50], direction, cfg)

  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


In [31]:
def print_direction_example(direction, n=5):
    print_top_examples(prompts, max_activation_per_prompt, direction, n)

# Max activations
interact(print_direction_example, 
         direction=IntSlider(min=0, max=encoder.d_hidden-1, step=1, value=0),
         n=IntSlider(min=1, max=20, step=1, value=5))


interactive(children=(IntSlider(value=0, description='direction', max=16383), IntSlider(value=5, description='…

<function __main__.print_direction_example(direction, n=5)>

In [32]:
# Activations of different directions on the same token
prompt = "One day, a little girl named Lily went for a walk in the park"
acts = get_acts(prompt, model, encoder, cfg)[-1] # d_enc
print(f"Active directions on last token: {acts.nonzero().shape[0]} out of {acts.shape[0]}")
active_directions = acts.nonzero().squeeze(1)
highly_active_directions = torch.argwhere((acts > 0.5)).squeeze(1)
low_active_directions = torch.argwhere((acts < 0.5) & (acts > 0.1)).squeeze(1)
px.histogram(acts.cpu().numpy(), title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for prompt", histnorm="probability", nbins=40)

Active directions on last token: 51 out of 16384


In [33]:
for active_direction in low_active_directions[:5]:#active_directions[:10]:
    print(f"Direction {active_direction}")
    print_top_examples(prompts, max_activation_per_prompt, active_direction, 2)

Direction 571


Direction 996


Direction 1720


Direction 1878


Direction 1965


In [55]:
layer = 0

def get_token_kurtosis_for_feature(model: HookedTransformer, decoder_feature: torch.Tensor):
    W_out = model.W_out[layer]
    resid_dirs = torch.nn.functional.normalize(decoder_feature @ W_out, dim=-1)
    unembed = torch.nn.functional.normalize(model.unembed.W_U, dim=0)
    sims = einops.einsum(resid_dirs, unembed, 'd_model, d_model d_vocab -> d_vocab')

    mean = torch.mean(sims)
    variance = torch.mean((sims - mean) ** 2)
    std = torch.sqrt(variance)
    excess_kurt = torch.mean(((sims - mean) / std) ** 4) - 3
    return excess_kurt

def get_token_kurtosis_for_decoder(model: HookedTransformer, decoder: torch.Tensor):
    '''Return mean kurtosis over all decoder features' cosine sims with the unembed (higher is better)'''
    W_out = model.W_out[layer]
    resid_dirs = torch.nn.functional.normalize(decoder @ W_out, dim=-1)
    unembed = torch.nn.functional.normalize(model.unembed.W_U, dim=0)
    cosine_sims = einops.einsum(resid_dirs, unembed, 'd_hidden d_model, d_model d_vocab -> d_hidden d_vocab')
    
    mean = einops.repeat(cosine_sims.mean(dim=-1), f'd_hidden -> d_hidden {cosine_sims.shape[1]}')
    std = einops.repeat(cosine_sims.std(dim=-1), f'd_hidden -> d_hidden {cosine_sims.shape[1]}')
    kurt = torch.mean((cosine_sims - mean / std) ** 4, dim=-1) - 3
    return kurt

def top_boosted_tokens(model: HookedTransformer, decoder_feature: torch.Tensor, k=10, plot=False):
    W_out = model.W_out[layer]
    resid_dirs = decoder_feature @ W_out
    tokens = resid_dirs @ model.unembed.W_U

    all_ignore, _ = haystack_utils.get_weird_tokens(model, plot_norms=False)
    values, tokens = haystack_utils.top_k_with_exclude(tokens, k, exclude=all_ignore)
    boosted_labels = model.to_str_tokens(tokens)

    if plot:
        assert k < 300, "Too many tokens to plot"
        fig = haystack_utils.line(x=values.cpu().numpy(), xticks=boosted_labels, title=f"Boosted tokens", width=1200)
        fig.show()

    return boosted_labels

scores = get_token_kurtosis_for_decoder(model, encoder.W_dec)

top_k = torch.topk(scores.abs(), k=10)
for i in top_k.indices:
    print(top_boosted_tokens(model, encoder.W_dec[i]))

[' birth', ' existed', ' pilots', ' startling', ' furthe', 'luck', 'TO', ' leaders', ' abl', ' sensations']
[').', 'End', 'pot', ' showcase', ' deceive', '.)', 'cloud', 'found', ' teleport', ' fools']
[' defined', 'QU', ',', ' prof', ' honored', ' emotionally', ' connection', ' Brothers', ' unthinkable', ' throats']
[' arrests', ' collapses', ' seven', ' collapsed', 'cca', ' cured', ' exploded', ' worst', ' seeks', ' emits']
['lands', 'ho', 'es', 'nce', 'y', 'rich', 'land', ' salon', 'paper', ' estate']
[' pac', ' subsequent', 'time', ' goof', 'Look', ' horrible', ' willingness', ' prank', 'cing', ' tone']
[' reference', ' non', ' base', ' stripping', ' sealing', ' treacherous', ' +', ' valves', ' Sly', ' dis']
[' greater', ' inner', ' tro', ' Higher', ' England', ' spectacular', ' gotta', 'lich', ' composed', ' camping']
[' don', ' questions', 'giving', ' borrowing', ' privilege', ' execute', 'adoes', 'top', ' may', 'Ye']
[' brittle', 'exit', ' Testing', 'j', 'coming', ' unreliable', 

In [ ]:
for active_direction in highly_active_directions:#active_directions[:10]:
    print(f"Direction {active_direction}")
    print_top_examples(prompts, max_activation_per_prompt, active_direction, 4)

Direction 16335


Direction 3680


Direction 15070


Direction 7357


Direction 10620


In [ ]:
token = model.to_single_token(" park")
token_prompts = []
for prompt in prompts[:1000]:
    tokens = model.to_tokens(prompt)
    if token in tokens:
        token_prompts.append(prompt)
print(len(token_prompts))


163


In [ ]:
for direction in active_directions[:10]:
    plot_direction_frequency(prompts[:50], direction, cfg)

  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

...

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


In [ ]:
for prompt in token_prompts[:2]:
    for direction in active_directions[:10]:
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [ ]:
# direction = 9000
# fig = px.histogram(max_activation_per_prompt[:, direction].tolist(), title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for direction {direction}", histnorm="probability")
# fig.update_layout(
#     xaxis_title="Activation",
#     yaxis_title="Probability",
#     width = 800,
#     showlegend=False
# )
# fig.update_yaxes(type='log')
# fig.show()
# print_top_examples(prompts, max_activation_per_prompt, direction)

In [ ]:
# Look for active features on specific tokens in prompt
# Baseline: look at neurons
# Train with bigger  L1s
# At some point, it should become non monosemantic as it can just copy MLP
# Train without L1, see what happens